In [1]:
import psycopg2

import pandas as pd
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from copy import deepcopy

### parse sehir contacts directory (full names, emails)

In [47]:
contacts = pd.read_csv('datasets/contacts.csv', 
                       encoding = "ISO-8859-1", 
                       usecols=['First Name', 'Last Name', 'Primary Email']).dropna()

In [108]:
def get_matches_edit_distance(item, choices, limit, scorer=fuzz.token_sort_ratio):
    return process.extract(item, choices, limit=limit, scorer=scorer)

def get_sehir_twitter_matches(twitter_users, sehir_directory, limit=2):
    fullnames = [' '.join(first_last_name) 
                 for first_last_name in sehir_directory[['First Name', 'Last Name']].values]
    
    twitter_user_by_screen_name = twitter_users.set_index('twitter_screen_name')
    
    for twitter_screen_name in twitter_users['twitter_screen_name']:
        twitter_name = twitter_user_by_screen_name.loc[twitter_screen_name, 'twitter_name']
        sehir_matches = get_matches_edit_distance(twitter_name, fullnames, limit)
        
        yield (twitter_screen_name, sehir_matches)
        
def filter_matches_by_threshold(matches_dict, threshold=70):
    filtered_dict = dict()
    for twitter_screen_name, matches in matches_dict.items():
        filtered = [(match, score) for match, score in matches if score > threshold]
        
        if filtered:
            filtered_dict[twitter_screen_name] = filtered
        
    return filtered_dict

def get_matches_dataframe(twitter_users, sehir_directory, threshold, limit):
    matches = {screen_name : sehir_matches for screen_name, sehir_matches in 
               get_sehir_twitter_matches(twitter_users, sehir_directory, limit=limit)}
    
    filtered_matches = filter_matches_by_threshold(matches, threshold=threshold)
    screen_names = filtered_matches.keys()
    return pd.DataFrame({'twitter_screen_name': list(screen_names),
                         'sehir_matches': [filtered_matches[screen_name] for screen_name in screen_names]})

### connect to sql database

In [80]:
connection = psycopg2.connect("dbname=twitter_accounts user=postgres")

twitter_users = pd.read_sql("SELECT * FROM twitter_user", connection)\
.rename(columns={'id': 'GUID', 
                 'name': 'twitter_name',
                 'description': 'profile_description',
                 'screen_name': 'twitter_screen_name'})

user_connections = pd.read_sql("SELECT * FROM twitter_connection", connection).drop('id', axis=1)

In [81]:
twitter_users.head(2)

,GUID,twitter_name,twitter_screen_name,profile_description,followers_count,friends_count,favourites_count,statuses_count,lang
0,106086098,ŞEHİR Üniversitesi,SehirUniversite,İstanbul Şehir Üniversitesi'nin Resmi Twitter ...,11367,165,387,3346,tr
1,813478272910630913,SehirCHES,SehirCHES,Official Twitter for @SehirUniversite 's Cente...,102,201,27,44,tr


### user_connections contains follower/followee relationships by GUID

In [82]:
user_connections.head(2)

,from_user_id,to_user_id
0,106086098,813478272910630913
1,106086098,835028362032742400


### matching subset of sehir twitter accounts (first 5000)

In [111]:
sehir_matches_df = get_matches_dataframe(twitter_users[:5000],
                                         sehir_directory, threshold=90, limit=2)

merged = sehir_matches_df.merge(twitter_users, left_on='twitter_screen_name', right_on='twitter_screen_name')

In [116]:
merged[['GUID', 'twitter_screen_name', 'twitter_name', 'sehir_matches']].tail(5)

,GUID,twitter_screen_name,twitter_name,sehir_matches
330,413849104,mustafacelen70,Mustafa ÇELEN,"[(Mustafa Celen, 96)]"
331,429101069,hamitokur,hamitokur,"[(Hamit Okur, 95)]"
332,3624969136,hasan_atan,Hasan ATAN,"[(Hasan Atan, 100)]"
333,397251795,zeynpchn,Zeynep Cihan,"[(Zeynep Cihan, 100), (Zeynep Ilhan, 92)]"
334,3234142326,MCanErim,Mehmet Can Erim,"[(Mehmet Can Erim, 100)]"


In [117]:
merged.to_csv('datasets/sehir_matches.csv', index=False)